# JupyterHub Notebook

### This notebook server is hosted on the OpenShift platform which provides a separate server for each individual user. The platform takes care of the provisioning of the server and allocating related to storage.

### First, install and import required libraries and watermark our file - to show what libraries and versions we're using. 

In [15]:
!pip install watermark seaborn pandas sklearn category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 KB 5.3 MB/s eta 0:00:00


In [31]:
import os



In [32]:
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
# import pandas as pd
# import modin.pandas as pd
from datetime import datetime
import watermark
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


import os
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline


# import tools as tools
%matplotlib inline
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [33]:
%watermark -n -v -m -g -iv


Python implementation: CPython
Python version       : 3.8.8
IPython version      : 8.0.0rc1

Compiler    : GCC 8.5.0 20210514 (Red Hat 8.5.0-3)
OS          : Linux
Release     : 4.18.0-305.30.1.el8_4.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Git hash: 5aa16f12e698fbbf9106ac552364537849ec7f7c

watermark        : 2.3.0
numpy            : 1.19.2
matplotlib       : 3.4.1
joblib           : 1.1.0
category_encoders: 2.3.0
sklearn          : 0.0
pandas           : 1.2.4
seaborn          : 0.11.2



### Read your file 

In [34]:

data = pd.read_csv('../data/data.csv')
data.head(5)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,148,Male,0,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,45.65,45.65,Yes
1,463,Male,0,Yes,Yes,4,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,No,Electronic check,101.15,385.90,Yes
2,471,Female,1,No,No,17,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,20.65,330.60,No
3,496,Male,0,No,No,22,No,No phone service,DSL,No,...,Yes,No,No,Yes,One year,Yes,Bank transfer (automatic),43.75,903.60,Yes
4,833,Female,0,Yes,Yes,70,Yes,No,DSL,Yes,...,Yes,Yes,No,Yes,One year,No,Credit card (automatic),74.10,5222.30,No


In [35]:
# Convert binary variable into numeric so plotting is easier. We need to later take mean
data['Churn'] = data['Churn'].map({'Yes': 1, 'No': 0})

In [36]:
data.replace(" ", np.nan, inplace=True)


In [37]:
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])

In [38]:
mean = data['TotalCharges'].mean()
data.fillna(mean, inplace=True)
# Now we know that total charges has nan values
data.isna().sum()



customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [39]:
import category_encoders as ce
import joblib

names = ['gender', 'Partner', 'Dependents', 'PhoneService', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
# for column in names:
#     labelencoder(column)
data_enc = data
data_enc = data_enc.drop(['Churn', 'customerID'], axis=1)
data_enc.head(1)
enc = ce.ordinal.OrdinalEncoder(cols=names)
enc.fit(data_enc)
joblib.dump(enc, '../deploy_model/ocp/CustomerChurnOrdinalEncoder.pkl')

labelled_set = enc.transform(data_enc)
labelled_set.tail(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
7038,1,0,1,1,1,1,No,No,No internet service,No internet service,No internet service,No internet service,3,3,Month-to-month,1,Mailed check,18.85,18.85
7039,2,0,1,1,1,1,Yes,Fiber optic,No,No,No,No,1,1,Month-to-month,1,Electronic check,74.50,74.50
7040,1,0,2,1,69,1,No,DSL,Yes,Yes,No,No,1,1,One year,1,Credit card (automatic),53.65,3804.40
7041,1,0,2,2,70,1,Yes,DSL,No,Yes,Yes,Yes,2,2,Two year,1,Electronic check,84.10,5979.70
7042,2,0,1,1,3,1,No,DSL,Yes,Yes,No,Yes,1,2,Two year,2,Mailed check,71.10,213.35


In [40]:

names = ['MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport']

ohe = ce.OneHotEncoder(cols=names)
data_ohe = data
data_ohe = data_ohe.drop(['Churn', 'customerID'], axis=1)
data_ohe.head(1)
ohe.fit(data_ohe)
joblib.dump(ohe, '../deploy_model/ocp/CustomerChurnOneHotEncoder.pkl')
final_set = ohe.transform(labelled_set)
final_set.tail(5)
labelled_set.shape

(7043, 19)

In [41]:
labels = data['Churn']
X_train, X_test, y_train, y_test = train_test_split(final_set, labels, test_size=0.2)
print ('Training Data Shape',X_train.shape, y_train.shape)
print ('Testing Data Shape',X_test.shape, y_test.shape)

Training Data Shape (5634, 36) (5634,)
Testing Data Shape (1409, 36) (1409,)


In [42]:

Y = data['Churn']
X = final_set



### In this next section, we define the method train_and_save_model() where we train and then push our model to Verta - for visualisation 

In [43]:
def train_and_save_model():
    kfold = KFold(n_splits = 3)
    model = DecisionTreeClassifier(max_depth=5, criterion='gini',min_samples_leaf = 3 ,min_samples_split = 10)
    model = model.fit(X_train, y_train)
    joblib.dump(model, '../deploy_model/ocp/CustomerChurnPredictor.sav')
    results = model_selection.cross_val_score(model,X,Y,cv = kfold)
    print(results)
    print('Accuracy',results.mean()*100)


    return model

### In this next section, we define the method explain_model(), where we make available an *_explanation_* of the reasons the model made the decisions it did. This is very useful for auditing purposes as well as for the Application development consumers of the model - who can optionally expand and utilise these reasons for their purposes.

In [44]:
# from alibi.utils.data import gen_category_map

# def explain_model(model, X_train, X_test_record):
#     fnames = X_train.columns.tolist()
#     predict_fn = lambda x: model.predict_proba(x)
#     explainer = AnchorTabular(predict_fn, fnames)
#     explainer = explainer.fit(X_train.values, disc_perc=[25, 50, 75])
#     explanation = explainer.explain(X_test_record.values[0])
#     print('Anchor: %s' % explanation['anchor'])
#     print('Precision: %.2f' % explanation['precision'])
#     print('Coverage: %.2f' % explanation['coverage'])
#     return explainer

In [45]:
model = train_and_save_model()
# explainer = explain_model(model, X_train, X_test)
# with open("CustomerChurnPredictorAlibi.dill", "wb") as x_f:
#     dill.dump(explainer, x_f)

[0.7802385  0.8032368  0.78909246]
Accuracy 79.08559188612233


In [46]:
# minioClient = get_s3_server()
# minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/CustomerChurnPredictor.sav' , file_path='./CustomerChurnPredictor.sav')
# # minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/CustomerChurnPredictorAlibi.dill' , file_path='./CustomerChurnPredictorAlibi.dill')
# minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/CustomerChurnOrdinalEncoder.pkl' , file_path='./CustomerChurnOrdinalEncoder.pkl')
# minioClient.fput_object(bucket_name='models', object_name=experiment_id  +'/CustomerChurnOneHotEncoder.pkl' , file_path='./CustomerChurnOneHotEncoder.pkl')


In [47]:
print('Notebook complete')

Notebook complete
